In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error

In [ ]:

import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
from torch.utils.data import random_split
from torch.optim import SGD, Adam
from torch.utils.data import DataLoader
from torchinfo import summary

In [ ]:
def seed_everything(seed=2024):
    import random
    import os
    import numpy as np
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything()

In [ ]:
def load_data(path_to_data: str):
    data = pd.read_csv(path_to_data)
    return data

## 1. Regression Model

The following code block constructs two regression models to predict the salaries of baseball players using the `Hitters Dataset` (from the ISLR Book). We will use `LinearRegression`and `Lasso`. Complete the cell

In [ ]:
hitters = load_data(path_to_data='data/hitters.csv').dropna()
hitters = pd.get_dummies(hitters, columns=['League', 'Division', 'NewLeague'], drop_first=True)
hitters = hitters.drop('Names', axis=1)

X = #TODO
y = #TODO 


linear_pipeline = Pipeline([
    ('scaler', #TODO),
    ('reg', #TODO))
])

lasso_pipeline = Pipeline([
    ('scaler', #TODO,
    ('reg', #TODO
])


linear_pipeline.fit(#TODO)
lasso_pipeline.fit(#TODO)

linear_pred = linear_pipeline.predict(#TODO)
lasso_pred = lasso_pipeline.predict(#TODO)

linear_mae = mean_absolute_error(#TODO)
lasso_mae = mean_absolute_error(#TODO)
print(f'Linear MAE: {linear_mae}, Lasso MAE: {lasso_mae}')


## 2. Dataset in Pytorch

The cell below is for converting our **training and testing data into PyTorch tensors** and then creating a `TensorDataset` for each of them.

- `X_train_tensor = torch.tensor(X_train.astype(np.float32))` and `y_train_tensor = torch.tensor(y_train.astype(np.float32))`: These lines convert the training data (`X_train`) and labels (`y_train`) into PyTorch tensors. The `astype(np.float32)` part is used to ensure that the data is in the `float32` format, which is a requirement for many PyTorch functions.

- `hit_traindataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)`: This line creates a `TensorDataset` from the training data and labels. A `TensorDataset` is a dataset provided by PyTorch that wraps tensors. Each sample will be retrieved by indexing tensors along the first dimension.

The next two lines do the same for the **testing data (`X_test`) and labels (`y_test`)**.

See how the data are organized : `hit_traindataset[0]`

In [ ]:
X_train_tensor = torch.tensor(X_train.astype(np.float32))
y_train_tensor = torch.tensor(y_train.astype(np.float32))

hit_traindataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)

X_test_tensor = torch.tensor(X_test.astype(np.float32))
y_test_tensor = torch.tensor(y_test.astype(np.float32))

hit_testdataset = torch.utils.data.TensorDataset(X_test_tensor, y_test_tensor)

The cell below creates `DataLoader` objects for both the training and testing datasets. These DataLoader objects will be used to iterate over the datasets in batches during the training and testing phases of the model. This allows for more efficient use of memory and can make training faster, especially for large datasets. The DataLoader also handles the shuffling of the data, which helps ensure that the model doesn't learn any unintended patterns from the order of the training data.

In [ ]:
hit_trainloader = torch.utils.data.DataLoader(hit_traindataset, batch_size=32, shuffle=True)
hit_testloader = torch.utils.data.DataLoader(hit_testdataset, batch_size=32, shuffle=False)

We define new class named `HittersModel` which is a subclass of `nn.Module`. The `nn.Module` class is a fundamental class provided by PyTorch and is used to represent neural network models. 

In `__init__`, we define:

- `self.fc1 = nn.Linear(input_dim, 50)`: A fully connected layer with `input_dim` inputs and 50 outputs.
- `self.relu = nn.ReLU()`: The ReLU activation function.
- `self.dropout = nn.Dropout(0.4)`: A dropout layer with a rate of 0.4 for regularization.
- `self.fc2 = nn.Linear(50,1)`: Another fully connected layer with 50 inputs and 1 output for binary classification.

Additionally, the `__init__` method includes a call to `super().__init__()`. This function allows subclasses to access methods from their parent class. For instance, `nn.Module` has its own `__init__` method, which is different from the `__init__` method we've defined for `HittersModel`. By using `super().__init__()`, we're calling the `__init__` method of the parent class, ensuring that our model is correctly interpreted by PyTorch.


The `forward` method defines the forward pass of the neural network, which is how the network processes inputs.

Here's what each line does:

- `x = self.fc1(x)`: This line passes the input `x` through the first fully connected layer (`fc1`).

- `x = self.relu(x)`: This line applies the ReLU activation function to the output of the first fully connected layer. The ReLU function is defined as `ReLU(x) = max(0, x)`, which introduces non-linearity into the model.

- `x = self.dropout(x)`: This line applies dropout to the output of the ReLU function. Dropout randomly sets a fraction of input units to 0 at each update during training time, which helps prevent overfitting.

- `x = self.fc2(x)`: This line passes the output of the dropout layer through the second fully connected layer (`fc2`).

- `return x.squeeze()`: This line removes dimensions of size 1 from the output. For example, if the output shape was `(batch_size, 1)`, it becomes `(batch_size,)` after squeezing. This is often necessary because many loss functions expect the target and input to have the same dimensions.

In summary, the `forward` method takes an input tensor, passes it through two fully connected layers with a ReLU activation and dropout in between, and returns the output after squeezing unnecessary dimensions.


In [ ]:
class HittersModel(nn.Module):
    def __init__(self, input_dim):
        super(HittersModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 50)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.4)
        self.fc2 = nn.Linear(50,1)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x.squeeze()

## Training Loop

The `train_function` represents a standard training loop in Pytorch. It iterates over the dataset multiple times (epochs) and calculates updates.

In [ ]:
def train_function(model, train_loader, test_loader, epochs, optimizer, criterion):
    for epoch in range(epochs):
        model.train()
        epoch_tloss, epoch_vloss, epoch_vmetrics = 0., 0., 0.
        for X, y in train_loader: # X is the input and y is the target
            optimizer.zero_grad() # zero the parameter gradients
            output = model(X) 
            tloss = criterion(output, y)
            tloss.backward() # backpropagation
            optimizer.step() # update the weights
            epoch_tloss += tloss.item() # add the loss to the epoch loss
        model.eval()
        with torch.no_grad():
            for X, y in test_loader:
                output = model(X)
                vloss = criterion(output, y)
                mae = nn.functional.l1_loss(output, y) # mean absolute error
                epoch_vloss += vloss.item() 
                epoch_vmetrics += mae.item()
            print(f'Epoch: {epoch} : Train Loss: {epoch_tloss/len(train_loader):.2f} || Test Loss: {epoch_vloss/len(test_loader):.2f} || Test MAE: {epoch_vmetrics/len(test_loader):.2f}')

### Questions

1. What is the purpose of `model.train()` and `model.eval()`? What might go wrong if we forgot to call these functions?

2. Why do we compute the loss on both the training data and the testing data? What is the difference between these two losses?

3. Why do we use `with torch.no_grad():` during evaluation? What would happen if we didn't use it?

#### You can run the cell below to train the model on Hitters dataset

In [ ]:
model = HittersModel(input_dim=X_train.shape[1]) 
optimizer = Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()
train_function(model, hit_trainloader, hit_testloader, 30, optimizer, criterion)

# 2. CNN with Pytorch 

### Load FashionMNIST data

In [ ]:
from torchvision.datasets import FashionMNIST
from torchvision.transforms import (Compose,
                                    ToTensor, 
                                    Normalize)
(train_data,test_data )= [FashionMNIST(root="data", train=train, download=True, transform=ToTensor()) for train in [True, False]]
train_data

In [ ]:
# Explore the dataset
print(f"Number of training examples: {len(train_data)}")
print(f"Number of test examples: {len(test_data)}")
print(f"Classes: {train_data.classes}")
print(f"Data dimension: {train_data.data.shape}")


In [ ]:
# Show the first 10 images
classes_map = {idx:value for idx,value in zip(np.arange(len(train_data.classes)), train_data.classes)}
def show_image(data):
    fig = plt.figure(figsize=(10,5))
    for i in range(10):
        ax = plt.subplot(2, 5, i+1)
        im = data[i][0].permute(1,2,0).squeeze().numpy()
        plt.imshow(im)
        plt.title(classes_map[data[i][1]])
        plt.axis("off")
    plt.show()
show_image(train_data)


## Convolution 2D
The `Conv2d` function in PyTorch represents a 2-dimensional convolutional layer. Let's break down the parameters:

- `in_channels`: This parameter specifies the number of channels (or depth) in the input image or feature map. For example, if you're dealing with RGB images, `in_channels` would be 3 since there are three color channels (red, green, blue).

- `out_channels`: This parameter defines the number of filters (or kernels) in the convolutional layer. Each filter produces one channel in the output feature map. In your example, there will be 32 output channels.

- `kernel_size`: This parameter determines the size of the convolutional kernel. Here, it's set to `3`, indicating that each kernel will be 3x3.

- `padding`: Padding is added to the input image to ensure that the spatial dimensions of the output feature map are the same as the input. `padding=1` means that 1 pixel of padding will be added to all sides of the input.

- `stride`: Stride specifies the step size at which the kernel moves across the input image. Here, `stride=1` means the kernel moves one pixel at a time.

To compute the output size of the convolutional layer, you can use the following formulas:

Output height: 
$$ \text{output\_height} = \frac{{\text{input\_height} + 2 \times \text{padding} - \text{kernel\_size}}}{{\text{stride}}} + 1 $$

Output width: 
$$ \text{output\_width} = \frac{{\text{input\_width} + 2 \times \text{padding} - \text{kernel\_size}}}{{\text{stride}}} + 1 $$

Output channels: 
$$ \text{output\_channels} = \text{out\_channels} $$

So, if you have an input image with dimensions (input_channels, input_height, input_width), after passing through this convolutional layer, you will get an output feature map with dimensions (out_channels, output_height, output_width).

## Maxpool 2D

The `MaxPool2d` function in PyTorch represents a 2-dimensional max pooling layer. Let's break down the parameters:

- `kernel_size`: This parameter specifies the size of the window over which the maximum value is computed. For example, `kernel_size=2` means that a 2x2 window will be used to perform the max pooling operation.

Max pooling works by sliding the defined window (kernel) over the input feature map and selecting the maximum value within each window. This operation helps in downsampling the input feature map, reducing its spatial dimensions while retaining the most significant information.

## DropOut 

The `nn.Dropout` function in PyTorch represents a regularization technique to prevent overfitting. The parameter `p`specifies the probability of dropping out neurons duriung training. Dropout works by randomly setting the outputs of a fraction of neurons in the specified layer to zero during each training iteration. 



# Simple Convolution Layer
Below, we define a basic convolutional neural network for image classification, specifically for the FashionMNIST dataset. Please examine the input and output dimensions of each layer and understand how these sizes are calculated.

| Layer (type:depth-idx)  | Input Shape | Output Shape |
|-------------------------|-------------|--------------|
| Net                     | [1, 28, 28] | [1, 10]      |
| ├─Conv2d: 1-1           | [1, 28, 28] | [32, 28, 28] |
| ├─MaxPool2d: 1-2        | [32, 28, 28]| [32, 14, 14] |
| ├─Conv2d: 1-3           | [32, 14, 14]| [64, 14, 14] |
| ├─MaxPool2d: 1-4        | [64, 14, 14]| [64, 7, 7]   |
| ├─Dropout: 1-5          | [64, 7, 7]  | [64, 7, 7]   |
| ├─Linear: 1-6           | [64, 7, 7]  | [3136, 128]  |
| ├─Linear: 1-7           | [3136, 128] | [128, 10]    |


In [ ]:
class Net(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=input_channels,out_channels=32,kernel_size=3,padding=1,stride=1)
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,stride=1,padding=1)
        self.dropout = nn.Dropout(p=0.25)
        self.fc1 = nn.Linear(in_features=64*7*7,out_features=128)
        self.fc2 = nn.Linear(in_features=128,out_features=num_classes)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout(x)
        x = x.view(-1,64*7*7)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

We split the training data into train and validation set and create `Dataloader` object for each dataset. 
Complete the missing part 

In [ ]:
# Split the dataset into training and validation
dataset_size = len(train_data)
train_size = int(0.7 * dataset_size)  # 70% for training
val_size = dataset_size - train_size  # 30% for validation

train_dataset, val_dataset = random_split(train_data, [train_size, val_size])

In [ ]:
# Define some parameters to train the model
input_channels = #TODO
num_classes = #TODO 
learning_rate = #TODO
epochs = #TODO
patience = #TODO
batch_size = #TODO

# Create DataLoaders 
trainloader = #TODO
valloader = #TODO  
testloader = #TODO

### We use the `Trainer` class in `utilsDL` to train a classification model. 

In [ ]:
from utilsDL import Trainer

model = Net(input_channels, num_classes)
optimizer = Adam(model.parameters(), lr=learning_rate)
criterion  = nn.CrossEntropyLoss()
device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")

trainer = Trainer(model, trainloader, valloader, testloader, criterion, optimizer, device) 
train_loss, train_acc, val_loss, val_acc = trainer.train(epochs=epochs, patience=patience)

In [ ]:
Trainer.plot_metrics(train_loss, val_loss, train_acc, val_acc)

In [ ]:
#Evaluate on the test data
test_acc = trainer.evaluate()

# Your Task

Now, it's your turn to train a deep learning model. We will utilize the CIFAR-10 dataset available in the torchvision package. This dataset can be loaded by running the subsequent cell. You are required to configure the parameters for your model, select an optimizer and an learning rate, determine your batch size, and finally, present your results.

`Note`: Please pay attention to image shape, input/output of each layer

In [ ]:
from torchvision.datasets import CIFAR10
(cifar_train, cifar_test) = [CIFAR10(root="data", train=train, download=True, 
                                     transform=Compose([ToTensor(), 
                                                        Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))])) for train in [True, False]]